In [1]:
import numpy as np
import tensorflow as tf
import util
import numpy as np
import tensorflow as tf
import util
import torch
from keras.utils.np_utils import *
from keras.datasets import mnist
MNIST_CLASSIFIER_FROZEN_GRAPH ='/home/imi432_006/huangdengrong/GAN-Research/metrics_gan/metrics/classify_mnist_graph_def.pb'
INPUT_TENSOR = 'inputs:0'
OUTPUT_TENSOR = 'logits:0'
def calculate_labels(images,epoch,global_step):
    label_dict={}
    for i in range(10):
        label_dict[i]=0
    eval_images = tf.convert_to_tensor(images)
    y_logit = util.mnist_logits(eval_images, MNIST_CLASSIFIER_FROZEN_GRAPH, INPUT_TENSOR, OUTPUT_TENSOR)
    labels=tf.argmax(y_logit, 1)
    labels=tf.Session().run(labels)
    for data in labels:
        label_dict[data]+=1
    for i in range(10):
        label_dict[i]=label_dict[i]/len(images)
    max_value=max(label_dict.values())
    min_value=min(label_dict.values())
    print('epoch:%d   global_step:%d'%(epoch,global_step))
    print(label_dict)
    print('chazhi:%.8f'%(max_value-min_value))
    return label_dict,max_value-min_value

Using TensorFlow backend.


In [2]:
from __future__ import print_function, division

from keras.datasets import mnist
from keras.layers import Input, Dense, Reshape, Flatten, Dropout
from keras.layers import BatchNormalization, Activation, ZeroPadding2D
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.convolutional import UpSampling2D, Conv2D
from keras.models import Sequential, Model
from keras.optimizers import Adam

import matplotlib.pyplot as plt

import sys
import os
import numpy as np

class LSGAN():
    def __init__(self):
        self.img_rows = 28
        self.img_cols = 28
        self.channels = 1
        self.img_shape = (self.img_rows, self.img_cols, self.channels)
        self.latent_dim = 100

        optimizer = Adam(0.0002, 0.5)

        # Build and compile the discriminator
        self.discriminator = self.build_discriminator()
        self.discriminator.compile(loss='mse',
            optimizer=optimizer,
            metrics=['accuracy'])

        # Build the generator
        self.generator = self.build_generator()

        # The generator takes noise as input and generated imgs
        z = Input(shape=(self.latent_dim,))
        img = self.generator(z)

        # For the combined model we will only train the generator
        self.discriminator.trainable = False

        # The valid takes generated images as input and determines validity
        valid = self.discriminator(img)

        # The combined model  (stacked generator and discriminator)
        # Trains generator to fool discriminator
        self.combined = Model(z, valid)
        # (!!!) Optimize w.r.t. MSE loss instead of crossentropy
        self.combined.compile(loss='mse', optimizer=optimizer)

    def build_generator(self):

        model = Sequential()

        model.add(Dense(256, input_dim=self.latent_dim))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Dense(512))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Dense(1024))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Dense(np.prod(self.img_shape), activation='tanh'))
        model.add(Reshape(self.img_shape))

        model.summary()

        noise = Input(shape=(self.latent_dim,))
        img = model(noise)

        return Model(noise, img)

    def build_discriminator(self):

        model = Sequential()

        model.add(Flatten(input_shape=self.img_shape))
        model.add(Dense(512))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dense(256))
        model.add(LeakyReLU(alpha=0.2))
        # (!!!) No softmax
        model.add(Dense(1))
        model.summary()

        img = Input(shape=self.img_shape)
        validity = model(img)

        return Model(img, validity)

    def train(self, epochs, batch_size=128, sample_interval=50):

        # Load the dataset
        (X_train, _), (_, _) = mnist.load_data()

        # Rescale -1 to 1
        X_train = (X_train.astype(np.float32) - 127.5) / 127.5
        X_train = np.expand_dims(X_train, axis=3)

        # Adversarial ground truths
        valid = np.ones((batch_size, 1))
        fake = np.zeros((batch_size, 1))

        nb_batches = int(X_train.shape[0] / batch_size)
        global_step = 0
        steps=[]
        values=[]

        for epoch in range(epochs):

            for index in range(nb_batches):
                global_step += 1
                imgs = X_train[index * batch_size:(index + 1) * batch_size]
                # Sample noise as generator input
                noise = np.random.normal(0, 1, (batch_size, self.latent_dim))

                # Generate a batch of new images
                gen_imgs = self.generator.predict(noise)

                # Train the discriminator
                d_loss_real = self.discriminator.train_on_batch(imgs, valid)
                d_loss_fake = self.discriminator.train_on_batch(gen_imgs, fake)
                d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

                # ---------------------
                #  Train Generator
                # ---------------------

                g_loss = self.combined.train_on_batch(noise, valid)

                # Plot the progress
#                 print("epoch:%d step:%d [D loss: %f, acc.: %.2f%%] [G loss: %f]" % (epoch, global_step,d_loss[0], 100 * d_loss[1], g_loss))

                # If at save interval => save generated image samples
                if global_step % sample_interval == 0:
                    label_dict,value=self.mode_drop(epoch,global_step)
                    steps.append(global_step)
                    values.append(value)
#             plt.subplots(1, 1)
        plt.plot(steps,values)
        plt.xlim([0,40000])
        plt.ylim([0.0,1.0])
        plt.xlabel('steps')
        plt.ylabel('epochs')
        plt.tick_params(axis='both',which='major',labelsize=13)
        plt.legend(loc='lower right')
        if not os.path.isdir('images_lsgan'):
            os.mkdir('images_lsgan')
        plt.savefig("images_lsgan/mode_drop.png")
        plt.show()



    def sample_images(self, epoch,global_step):
        r, c = 10, 10
        noise = np.random.normal(0, 1, (r * c, self.latent_dim))
        gen_imgs = self.generator.predict(noise)

        # Rescale images 0 - 1
        gen_imgs = 0.5 * gen_imgs + 0.5

        fig, axs = plt.subplots(r, c)
        cnt = 0
        for i in range(r):
            for j in range(c):
                axs[i,j].imshow(gen_imgs[cnt, :,:,0], cmap='gray')
                axs[i,j].axis('off')
                cnt += 1
        if not os.path.isdir('images_lsgan'):
            os.mkdir('images_lsgan')
        fig.savefig("images_lsgan/epoch_%d_step_%d.png" % (epoch,global_step))
        plt.close()
    def mode_drop(self,epoch,global_step):
        r, c = 10, 1000
        noise = np.random.normal(0, 1, (r * c, self.latent_dim))
        gen_imgs = self.generator.predict(noise)
        # Rescale images 0 - 1
        gen_imgs = 0.5 * gen_imgs + 0.5
        label_dict,value=calculate_labels(gen_imgs,epoch,global_step)
        return label_dict,value

if __name__ == '__main__':
    gan = LSGAN()
    gan.train(epochs=30, batch_size=64, sample_interval=200)


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_1 (Flatten)          (None, 784)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 512)               401920    
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 512)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 256)               131328    
_________________________________________________________________
leaky_re_lu_2 (LeakyReLU)    (None, 256)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 257       
Total params: 533,505
Trainable params: 533,505
Non-trainable params: 0
_________________________________________________________________
____

/home/imi432_006/anaconda3/envs/tf/lib/python3.5/site-packages/keras/engine/training.py:490: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


Instructions for updating:
Use tf.gfile.GFile.
epoch:0   global_step:200
{0: 0.0005, 1: 0.0, 2: 0.0, 3: 0.0097, 4: 0.0115, 5: 0.0, 6: 0.0, 7: 0.0001, 8: 0.9782, 9: 0.0}
chazhi:0.97820000
epoch:0   global_step:400
{0: 0.0001, 1: 0.0, 2: 0.0002, 3: 0.0001, 4: 0.0449, 5: 0.0, 6: 0.0, 7: 0.0008, 8: 0.9539, 9: 0.0}
chazhi:0.95390000
epoch:0   global_step:600
{0: 0.0059, 1: 0.0, 2: 0.0002, 3: 0.0536, 4: 0.0425, 5: 0.0002, 6: 0.0, 7: 0.0443, 8: 0.8533, 9: 0.0}
chazhi:0.85330000
epoch:0   global_step:800
{0: 0.2466, 1: 0.0332, 2: 0.0344, 3: 0.0519, 4: 0.0002, 5: 0.0152, 6: 0.0, 7: 0.1555, 8: 0.4029, 9: 0.0601}
chazhi:0.40290000
epoch:1   global_step:1000
{0: 0.1636, 1: 0.0171, 2: 0.037, 3: 0.2102, 4: 0.1546, 5: 0.0602, 6: 0.0092, 7: 0.069, 8: 0.2533, 9: 0.0258}
chazhi:0.24410000
epoch:1   global_step:1200
{0: 0.0816, 1: 0.0024, 2: 0.0478, 3: 0.1784, 4: 0.1342, 5: 0.0244, 6: 0.0322, 7: 0.1345, 8: 0.261, 9: 0.1035}
chazhi:0.25860000
epoch:1   global_step:1400
{0: 0.0356, 1: 0.0186, 2: 0.0681, 3:

epoch:11   global_step:10600
{0: 0.097, 1: 0.1132, 2: 0.0663, 3: 0.12, 4: 0.1043, 5: 0.047, 6: 0.0712, 7: 0.1535, 8: 0.128, 9: 0.0995}
chazhi:0.10650000
epoch:11   global_step:10800
{0: 0.0991, 1: 0.0945, 2: 0.0701, 3: 0.112, 4: 0.1312, 5: 0.0482, 6: 0.0882, 7: 0.1378, 8: 0.1362, 9: 0.0827}
chazhi:0.08960000
epoch:11   global_step:11000
{0: 0.0912, 1: 0.0968, 2: 0.0769, 3: 0.1074, 4: 0.1385, 5: 0.0521, 6: 0.0612, 7: 0.1459, 8: 0.1306, 9: 0.0994}
chazhi:0.09380000
epoch:11   global_step:11200
{0: 0.083, 1: 0.1236, 2: 0.0659, 3: 0.119, 4: 0.109, 5: 0.0643, 6: 0.0811, 7: 0.1427, 8: 0.1245, 9: 0.0869}
chazhi:0.07840000
epoch:12   global_step:11400
{0: 0.1186, 1: 0.1009, 2: 0.0641, 3: 0.1057, 4: 0.1297, 5: 0.0432, 6: 0.0759, 7: 0.1415, 8: 0.1095, 9: 0.1109}
chazhi:0.09830000
epoch:12   global_step:11600
{0: 0.1007, 1: 0.1077, 2: 0.057, 3: 0.1043, 4: 0.1523, 5: 0.0388, 6: 0.0829, 7: 0.1425, 8: 0.1165, 9: 0.0973}
chazhi:0.11350000
epoch:12   global_step:11800
{0: 0.1015, 1: 0.101, 2: 0.0603, 

epoch:22   global_step:21200
{0: 0.0947, 1: 0.1061, 2: 0.0632, 3: 0.1036, 4: 0.1104, 5: 0.0639, 6: 0.0975, 7: 0.163, 8: 0.1018, 9: 0.0958}
chazhi:0.09980000
epoch:22   global_step:21400
{0: 0.0941, 1: 0.1041, 2: 0.0883, 3: 0.1041, 4: 0.1152, 5: 0.0484, 6: 0.081, 7: 0.1471, 8: 0.1083, 9: 0.1094}
chazhi:0.09870000
epoch:23   global_step:21600
{0: 0.0984, 1: 0.1331, 2: 0.0689, 3: 0.1021, 4: 0.114, 5: 0.0486, 6: 0.0968, 7: 0.132, 8: 0.1157, 9: 0.0904}
chazhi:0.08450000
epoch:23   global_step:21800
{0: 0.1151, 1: 0.1194, 2: 0.0658, 3: 0.1068, 4: 0.1129, 5: 0.0575, 6: 0.0914, 7: 0.1059, 8: 0.1273, 9: 0.0979}
chazhi:0.06980000
epoch:23   global_step:22000
{0: 0.1035, 1: 0.1171, 2: 0.0761, 3: 0.1088, 4: 0.1105, 5: 0.0554, 6: 0.0872, 7: 0.1483, 8: 0.1029, 9: 0.0902}
chazhi:0.09290000
epoch:23   global_step:22200
{0: 0.0878, 1: 0.1183, 2: 0.0513, 3: 0.0965, 4: 0.1236, 5: 0.0611, 6: 0.1061, 7: 0.1281, 8: 0.1139, 9: 0.1133}
chazhi:0.07680000
epoch:23   global_step:22400
{0: 0.0903, 1: 0.1356, 2: 0

No handles with labels found to put in legend.


<Figure size 640x480 with 1 Axes>